In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from CinePred.new_model import *

In [17]:
df = import_data('../raw_data/IMDb movies.csv')
ohe = GenreOHE()
df = preproc(df)
ohe.fit(df)
df = ohe.transform(df)
print(df.columns)
print(df.shape)
df.sort_values('budget', inplace=True)

Index(['year', 'duration', 'budget', 'worlwide_gross_income', 'date_sin',
       'date_cos', 'History', 'Thriller', 'Western', 'Film-Noir', 'Drama',
       'Horror', 'Biography', 'Family', 'Music', 'Sci-Fi', 'Adventure',
       'Action', 'Musical', 'Documentary', 'War', 'Crime', 'Romance', 'Sport',
       'Mystery', 'Comedy', 'Fantasy', 'Animation'],
      dtype='object')
(12762, 28)


In [18]:
print(type(df.shape[0] / 2))
mid = int(df.shape[0] / 2)
print(mid)

<class 'float'>
6381


In [19]:
df.sort_values(by='budget', inplace=True)
df.reset_index(inplace=True)

df.drop(columns='index', inplace=True)

In [20]:
df1 = df.iloc[:mid]
df2 = df.iloc[mid:]

In [21]:
print(df1.shape)
print(df2.shape)

(6381, 28)
(6381, 28)


In [22]:
from sklearn.model_selection import cross_val_score

In [26]:
def get_mae2(df):
    X = df.drop(columns=['worlwide_gross_income'])
    y = df['worlwide_gross_income']
    model = XGBRegressor()
    print(cross_val_score(model, X, y, cv=5))

In [27]:
get_mae2(df1)

[-0.31820655  0.01651192 -0.05121392  0.01898145 -0.23328855]


In [28]:
get_mae2(df2)

[-0.18335718 -0.08749438  0.01314635 -0.05189867 -1.07857028]
